In [ ]:
import pandas as pd 
from pyhive import hive
import time
# second hive server connection for pyhive operations
conn = hive.Connection(host="localhost", port=10000, username="ubuntu3")
cursor = conn.cursor()
import datetime
now = datetime.datetime.now()
print(now.strftime("%A")) # to name the json file saved
today = now.strftime("%A")
#today = "Tuesday"

In [ ]:
import json
data_loaded = json.load(open("days.txt"))
data_loaded

In [ ]:
data = {}
data[today] = {}
data[today]['DevicePer'] = {}
data

In [ ]:
twCount = 'SELECT count(*) FROM tuesday'
rtCount = 'SELECT sum(rtcount) FROM tuesday'
fvCount = 'SELECT sum(fvcount) FROM tuesday'

In [ ]:
cursor.execute(twCount)
processes_tweet_count = cursor.fetchone()


In [ ]:
cursor.execute(rtCount)
retweetCount = cursor.fetchone()

In [ ]:
cursor.execute(fvCount)
favoriteCount = cursor.fetchone()

In [ ]:
print("Processed Tweet Count : ", int(processes_tweet_count[0]))
print("ReTweet Count : ",int(retweetCount[0]))
print("Favorite Count : ",int(favoriteCount[0]))

In [ ]:
data = {'ProcessedTweet': [int(processes_tweet_count[0])] ,
        'RT': [int(retweetCount[0])],
         'FV' : [int(favoriteCount[0])]
        }
df = pd.DataFrame (data, columns = ['ProcessedTweet','RT','FV'])
df

In [ ]:
lengs = 'SELECT tweetlen FROM tuesday'
cursor.execute(lengs)
total_length = 0
for i in cursor.fetchall():
    total_length = total_length + int(i[0])

df['AvgLength'] = total_length / int(processes_tweet_count[0])
print("Ortalama tweet karakter uzunlugu {:.2f} ".format(total_length / int(processes_tweet_count[0])))

In [ ]:
devices = 'SELECT device, count(*) FROM tuesday GROUP BY  device SORT BY device ASC'
cursor.execute(devices)
devGroup = cursor.fetchall()
devs = {}
for i in devGroup:
    devs[i[0]] = i[1]

In [ ]:
isRetweeted = 'SELECT isrt, count(*) FROM tuesday GROUP BY  isrt'
cursor.execute(isRetweeted)
isrt = cursor.fetchall()
rts = {}
for i in isrt:
    rts[i[0]] = i[1]

In [ ]:
locations = 'SELECT location, count(*) FROM tuesday GROUP BY  location'
cursor.execute(locations)
locationGroup = cursor.fetchall()
locs = {}
for i in locationGroup:
    locs[i[0]] = i[1]
del locs[None]
sorted_locs = {k: v for k, v in sorted(locs.items(),reverse = True, key=lambda item: item[1])}

In [ ]:
print("Devices : " ,devs)
print("\nRT situations : " , rts)

In [ ]:
print("\nTop 25 Locations : ")
top_locs = {}
x = 25
for k,v in sorted_locs.items():
    if x > 0 :
        top_locs[k] = v
        print(k , v)
    x = x -1 

In [ ]:
print(top_locs)

In [ ]:
import geopandas as gpd
import geopy
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
import time
locator = Nominatim(user_agent="myGeocoder")

In [ ]:
loc_geocodes = {}
for key,value in top_locs.items():
    try:
        location = locator.geocode(key)
        print(location.address)
        print("Latitude = {}, Longitude = {}".format(location.latitude, location.longitude))
        print("\n")
        loc_geocodes[key] = [value,[location.latitude, location.longitude]]
        print(loc_geocodes)
        time.sleep(3)
    except:
        pass

In [ ]:
print(loc_geocodes) # top locations with latitude - longitude

### CLEAN VARIABLES

In [ ]:
data_loaded =  json.load(open("days.txt"))

In [ ]:
data[today]['ProcessedTweetCount'] = int(processes_tweet_count[0])
data[today]['RtCount'] = int(retweetCount[0])
data[today]['FvCount'] = int(favoriteCount[0])
data[today]['Length'] = float(total_length / int(processes_tweet_count[0]))

In [ ]:
retweet_percentage_rt = "{:.2f}".format((int(rts['Yes']) / sum(rts.values()))*100)
data[today]['RtPer'] = float(retweet_percentage_rt)

In [ ]:
device_percentage_android = "{:.2f}".format((int(devs['Android']) / sum(devs.values()) )*100)
device_percentage_iphone = "{:.2f}".format((int(devs['iPhone']) / sum(devs.values()) )*100)
device_percentage_webapp = "{:.2f}".format((int(devs['WebApp']) / sum(devs.values()) )*100)
device_percentage_others = "{:.2f}".format((int(devs['Other']) / sum(devs.values()) )*100)

data[today]['DevicePer']['Android'] = float(device_percentage_android)
data[today]['DevicePer']['iPhone'] = float(device_percentage_iphone)
data[today]['DevicePer']['WebApp'] = float(device_percentage_webapp)
data[today]['DevicePer']['Other'] = float(device_percentage_others)

In [ ]:
data[today]['Locations'] = loc_geocodes
print(loc_geocodes) # top locations with latitude - longitude

In [ ]:
print(data[today])

In [ ]:
print(data_loaded[today])

In [ ]:
data_loaded[today] = data[today]

In [ ]:
data_loaded

In [ ]:
json.dump(data_loaded, open("days.txt",'w'))

In [1]:
# ENES ÇAVUŞ - FINAL YEAR PROJECT